In [1]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
import random
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

In [2]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

list_of_action = {
    0: (0,0),
    1: (0,1),
    2: (0,2),
    3: (1,0),
    4: (1,1),
    5: (1,2),
    6: (2,0),
    7: (2,1),
    8: (2,2)
}

rev_action = {
    (0,0) :0,
    (0,1) :1,
    (0,2) :2,
    (1,0) :3,
    (1,1) :4,
    (1,2) :5,
    (2,0) :6,
    (2,1) :7,
    (2,2) :8
}

In [3]:
def find_aval_actions(state_of_game,act):
  possible_actions = []
  for i in range(9):
    if state_of_game[act[i]] == 0:
      possible_actions.append(i)
  return possible_actions

In [70]:
class DQN(nn.Module):

    def __init__(self, in_features, outputs, action_list):
        super(DQN, self).__init__()
        
        self.action_list = action_list
        #print('input size',in_features )
        
        
        self.head1 = nn.Linear(in_features, 128)
        self.head2 = nn.Linear(128,128)
        self.outputlayer = nn.Linear(128,outputs)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        #print('input x', x.shape, type(x))
        #print('x reshape',x.reshape(x.shape(0),-1))
        Batch_size,_,_,_ = x.shape
        
        #print('x reshape',x.reshape(Batch_size,-1).shape)
        
        x = F.relu(self.head1(torch.Tensor(x).reshape(Batch_size,-1)))
        x = F.relu(self.head2(x))
        x = self.outputlayer(x)
        
        return x
    
    def act(self, state):
        state_tensor = torch.as_tensor(state, dtype=torch.float32)
        #print('----------------------------------------------------------')
        #print('act', state_tensor.shape, state_tensor.unsqueeze(-1).shape )
        q_values = self.forward(state_tensor.unsqueeze(0))
        #possible_q_values = mask_possible_action*q_values
        #print('possible_q_values',possible_q_values)
        
        
        max_q_idx = torch.argmax(q_values, dim=1)[0]
        max_q_idx_item = max_q_idx.detach().item()
        #print('max_q_idx_item', max_q_idx_item, type(max_q_idx_item))
        #print('action list', self.action_list)
        #print('rev_action',rev_action)
        move = self.action_list[max_q_idx_item]
        
        action_idx = rev_action[move]
        
        return action_idx
    
        

In [71]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE_FREQ = 500

number_of_actions = 9
state_features = 18


env = TictactoeEnv()
replay_buffer = deque(maxlen=BUFFER_SIZE)
rew_buffer = deque([0.0], maxlen=10)

episode_reward = 0.0

#initialization network
policy_net = DQN(state_features, number_of_actions,list_of_action).to("cpu")
target_net = DQN(state_features, number_of_actions,list_of_action).to("cpu")
target_net.load_state_dict(policy_net.state_dict())
#target_net.eval()





<All keys matched successfully>

In [72]:
def setup_state(grid, agent_tag):
    #print('Player',agent_tag,'grid',grid)
    #print('grid', type(grid))
    grid_tensor = torch.as_tensor(grid, dtype=torch.float32)
    x_state = torch.zeros((3,3,2))
    if agent_tag == 'X':
        x_state[:,:,0] = torch.eq(grid_tensor,1).int()
        x_state[:,:,1] = torch.eq(grid_tensor,-1).int()
    else:
        x_state[:,:,0] = torch.eq(grid_tensor,-1).int()
        x_state[:,:,1] = torch.eq(grid_tensor,1).int()
    
    #print('agent', x_state[:,:,0])
    #print('opponent',x_state[:,:,1])
    x_state_array = x_state.numpy()
    return x_state_array
    

In [73]:
Turns = np.array(['X','O'])
MIN_REPLAY_SIZE = 10000

#Initialize the Replay Buffer
replay_buffer = deque(maxlen=BUFFER_SIZE)

games = 0
episode_reward = 0.0

while(len(replay_buffer) < BATCH_SIZE):

    env.reset()
    #env.render()
    player1 = "X"
    player2 = "O"
    
    if games%2 == 0:
        player_opt = OptimalPlayer(epsilon = 1., player = player2)
        player_rnd = OptimalPlayer(epsilon = 1., player = player1)
        
    if games%2 == 1:

        #change the state by *-1 when optimal player is X
        player_opt = OptimalPlayer(epsilon = 0.5, player = player1)
        player_rnd = OptimalPlayer(epsilon = 1., player = player2)
        grid, _, __ = env.observe()
        comp_move = player_opt.act(grid)
        env.step(comp_move)
    

    
    for j in range(5):

       
        #random agent play
        obs = env.observe()[0]
        state = setup_state(obs,player_rnd.player)
        move = player_rnd.act(obs)
        grid, end, winner = env.step(move, print_grid=False)
        action = rev_action[move]
        #print('action', action
        
        
        if not end:
            #optimal player play
            move = player_opt.act(grid)
            grid, end, winner = env.step(move, print_grid=False)

        
        new_obs = env.observe()[0]
        new_state = setup_state(new_obs,player_rnd.player)
       
    
        #transition set-off
        rew = env.reward(player=Turns[1])
        transition = (state, action, rew, end, new_state)
        #print('transition', transition)
        replay_buffer.append(transition)
           
        print('games', games, 'buffer', len(replay_buffer))
        games += 1
        
        if end:
            #print('-------------------------------------------')
            #print('Game end, winner is player ' + str(winner))
            #print('Optimal player = ' +  Turns[0])
            #print('Random player = ' +  Turns[1])
            #env.render()
            env.reset()
            break




#for _ in range(MIN_REPLAY_SIZE):
    
   

games 0 buffer 1
games 1 buffer 2
games 2 buffer 3
games 3 buffer 4
games 4 buffer 5
games 5 buffer 6
games 6 buffer 7
games 7 buffer 8
games 8 buffer 9
games 9 buffer 10
games 10 buffer 11
games 11 buffer 12
games 12 buffer 13
games 13 buffer 14
games 14 buffer 15
games 15 buffer 16
games 16 buffer 17
games 17 buffer 18
games 18 buffer 19
games 19 buffer 20
games 20 buffer 21
games 21 buffer 22
games 22 buffer 23
games 23 buffer 24
games 24 buffer 25
games 25 buffer 26
games 26 buffer 27
games 27 buffer 28
games 28 buffer 29
games 29 buffer 30
games 30 buffer 31
games 31 buffer 32
games 32 buffer 33
games 33 buffer 34
games 34 buffer 35
games 35 buffer 36
games 36 buffer 37
games 37 buffer 38
games 38 buffer 39
games 39 buffer 40
games 40 buffer 41
games 41 buffer 42
games 42 buffer 43
games 43 buffer 44
games 44 buffer 45
games 45 buffer 46
games 46 buffer 47
games 47 buffer 48
games 48 buffer 49
games 49 buffer 50
games 50 buffer 51
games 51 buffer 52
games 52 buffer 53
games 53 buf

In [74]:
print(len(replay_buffer))

66


In [75]:
print(replay_buffer[0])

(array([[[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]], dtype=float32), 5, 0, False, array([[[0., 0.],
        [0., 0.],
        [0., 1.]],

       [[0., 0.],
        [0., 0.],
        [1., 0.]],

       [[0., 0.],
        [0., 0.],
        [0., 0.]]], dtype=float32))


In [77]:
#Main Training Loop

env.reset()
env.render()
player_opt = OptimalPlayer(epsilon=0.5, player='O')
agent_tag = 'X'

optimizer = torch.optim.Adam(policy_net.parameters(), lr=5e-4)
rew_buffer = deque([0.0], maxlen=100)
bad_pred_buffer = deque([0.0], maxlen=100)
random_buffer = deque([0.0], maxlen=100)

game = 0
old_game = 0
step = 0
bad_prediction = False

episode_reward = 0.0
episode_bad_predict = 0.0
episode_random = 0.0

criterion = torch.nn.HuberLoss(delta=1.0)

while(game < 2500):
    step +=1
    epsilon = np.interp(step, [0, EPS_DECAY], [EPS_START,EPS_END])
    
    rnd_sample = random.random()
    #print('rnd_sample', rnd_sample)
    
   
    #random agent play
    obs = env.observe()[0]
    state = setup_state(obs,agent_tag)
    available_action = find_aval_actions(obs, list_of_action)
    if rnd_sample <= epsilon:
        random_int = random.randint(0,len(available_action)-1)
        #print('random_int', random_int, 'len available action',len(available_action))
        move = available_action[random_int]
        #print('move',move)
        episode_random +=1
    else:
        move = policy_net.act(state)
        #print('move',move)
    
    
    
    if not move in available_action:
        #print(' bad prediction', available_action)
        bad_prediction = True
        
    
    
    if not bad_prediction:
        #print('env step')
        grid, end, winner = env.step(move, print_grid=False)
        #env.render()
        
    action = move
    #print('###### action #####',action)
    #env.render()
    
    if (not end) and (not bad_prediction):
        #optimal player play
        move = player_opt.act(grid)
        #print('optimal move', move)
        
        grid, end, winner = env.step(move, print_grid=False)
        #env.render()
    

    #transition set-off
    new_obs = env.observe()[0]
    new_state = setup_state(new_obs,agent_tag)
    if not bad_prediction:
        rew = env.reward(player='X')
        transition = (state, action, rew, end, new_state)
    else:
        rew = -1
        transition = (state, action, rew, True, new_state)
        #end = False
    
    #print('transition', transition)
    replay_buffer.append(transition)
    
    
    if end or bad_prediction:
        #print('game finish')
        if bad_prediction:
            episode_bad_predict += 1
            
        bad_prediction = False
        #print('-------------------------------------------')
        #print('Game end, winner is player ' + str(winner))
        #print('Optimal player = ' +  Turns[0])
        #print('Random player = ' +  Turns[1])
        #env.render()
        env.reset()
        episode_reward += rew
        game += 1
        if game % 10 == 0:
            
            rew_buffer.append(episode_reward)
            bad_pred_buffer.append(episode_bad_predict)
            random_buffer.append(episode_random)

            # Logging
            if game % 100 == 0:
                print()
                print('game', game)
                print('episode random',episode_random)
                print('Avg random', np.mean(random_buffer))
                print('episode bad predi',episode_bad_predict)
                print('Avg bad predi', np.mean(bad_pred_buffer))
                print('episode rew',episode_reward)
                print('Avg Rew', np.mean(rew_buffer)) 
                print('Loss', loss)
            episode_reward = 0.0
            episode_bad_predict = 0.0
            episode_random = 0.0
    
    
    # Start Gradient Step
    #print('replay_buffer', len(replay_buffer))
    transitions = random.sample(replay_buffer, BATCH_SIZE)
    
    obses = np.asarray([t[0] for t in transitions])
    #print('obses', type(obses))
    #print(obses)
    actions = np.asarray([t[1] for t in transitions])
    #print('actions type', type(actions))
    #print(actions)
    rews = np.asarray([t[2] for t in transitions])
    dones = np.asarray([t[3] for t in transitions])
    new_obses = np.asarray([t[4] for t in transitions])
    
    obses_tensor = torch.as_tensor(obses, dtype=torch.float32)
    actions_tensor = torch.as_tensor(actions, dtype=torch.int64).unsqueeze(-1)
    rews_tensor = torch.as_tensor(rews, dtype=torch.float32).unsqueeze(-1)
    dones_tensor = torch.as_tensor(dones, dtype=torch.float32).unsqueeze(-1)
    new_obses_tensor = torch.as_tensor(new_obses, dtype=torch.float32)
    
    #print('obses_tensor', obses_tensor.shape)
    #print('new_obses_tensor', new_obses_tensor.shape)
    
    # Compute Targets
    target_q_values = target_net(new_obses_tensor)
    #print('target_q_values',target_q_values.shape)
    #mask_possible_action = torch.ne(new_obses_tensor,0)*-1
    #print('mask_possible_action',mask_possible_action.shape)
    #possible_target_q_values = target_q_values * mask_possible_action
    #print('possible_target_q_values',possible_target_q_values.shape)
    max_target_q_values = target_q_values.max(dim=1, keepdim=True)[0]
    #print('max_target_q_values', max_target_q_values.shape)
    
    targets = rews_tensor + GAMMA * ( 1 - dones_tensor)  * max_target_q_values
    
    
    
    # Compute Loss
    
    q_values = policy_net(obses_tensor)
    
    
    # give the q values of the action we took
    action_q_values = torch.gather(input=q_values, dim=1, index=actions_tensor)
    
#     print('action_q_values',action_q_values.shape, 'targets',targets.shape)
#     print('actions_tensor', actions_tensor[0], actions_tensor[1])
#     print('q_values', q_values[0])
#     print('q_values row 1', q_values[1])
#     print('action_q_values',action_q_values[0])
    
    #loss = nn.functional.smooth_l1_loss(q_values_max, targets)
    loss = criterion(action_q_values, targets)
   
    
    #print('loss', loss, loss.shape, type(loss))
#     max_q_idx = torch.argmax(possible_q_values, dim=1)[0]
#     max_q_idx_item = max_q_idx.detach().item()
    
    
    # Gradient Descent
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Update Target Network
    if step % TARGET_UPDATE_FREQ == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
    # Logging
    


|- - -|
|- - -|
|- - -|


game 100
episode random 1.0
Avg random 9.454545454545455
episode bad predi 0.0
Avg bad predi 2.0
episode rew 1.0
Avg Rew -2.090909090909091
Loss tensor(0.0591, grad_fn=<HuberLossBackward0>)

game 200
episode random 4.0
Avg random 5.761904761904762
episode bad predi 2.0
Avg bad predi 1.5714285714285714
episode rew 2.0
Avg Rew -2.2857142857142856
Loss tensor(0.0508, grad_fn=<HuberLossBackward0>)

game 300
episode random 3.0
Avg random 4.548387096774194
episode bad predi 1.0
Avg bad predi 1.5161290322580645
episode rew -4.0
Avg Rew -2.3548387096774195
Loss tensor(0.0700, grad_fn=<HuberLossBackward0>)

game 400
episode random 2.0
Avg random 3.975609756097561
episode bad predi 0.0
Avg bad predi 1.4878048780487805
episode rew 5.0
Avg Rew -1.6341463414634145
Loss tensor(0.0691, grad_fn=<HuberLossBackward0>)

game 500
episode random 0.0
Avg random 3.450980392156863
episode bad predi 1.0
Avg bad predi 1.3529411764705883
episode rew 3.0
Avg Rew -1.1372549019607843
Loss 

In [41]:
obs


array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [32]:
a = np.zeros((3,3))

In [34]:
a[1,1] = 1
a[2,2] = -1
print(a)

[[ 0.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0. -1.]]


In [73]:
a_state = setup_state(a,'X')

In [74]:
print(a_state)

[[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [1. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 1.]]]


In [53]:
a_state.shape

(3, 3, 2)

In [56]:
a_state=a_state.reshape(9,2)
print(a_state.shape)

(9, 2)


In [76]:
print(a_state)
print(a_state.shape)
print(torch.Tensor(a_state).view(18))


[[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [1. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 1.]]]
(3, 3, 2)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.])


In [77]:
print(a_state.reshape(-1))

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1.]


In [39]:
b = (a_state[:,:,0]==a_state[:,:,1])
print(b,b.shape)

[[ True  True  True]
 [ True False  True]
 [ True  True False]] (3, 3)


In [47]:
b = b.astype(int)
print(b)

[[1 1 1]
 [1 0 1]
 [1 1 0]]
